In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import re
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import datetime as dt
from dateutil.parser import *
import random
import time
import json
from numpy import argmax
import warnings
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split,cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, roc_auc_score,classification_report, roc_auc_score,accuracy_score, \
precision_score, f1_score, recall_score, roc_curve
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import re
import seaborn as sns
from imblearn.over_sampling import SMOTE 

from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier, AdaBoostClassifier
from xgboost import XGBClassifier

%config InlineBackend.figure_format = 'retina'
plt.rc('xtick', labelsize=12) 
plt.rc('ytick', labelsize=12)

C:\Users\Jayy Jangam\.conda\envs\dsi\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
df_regression = pd.read_csv('../data/merged_train_final.csv')
df_multi = pd.read_csv('../data/merged_train_multi.csv')

In [3]:
test_df_regression = pd.read_csv('../data/merged_test_final.csv')
test_df_multi = pd.read_csv('../data/merged_test_multi.csv')

In [4]:
x = df_multi.drop(['wnvpresent'],axis=1)
y = df_multi['wnvpresent']

y.value_counts(normalize=True)

0    0.948107
1    0.051893
Name: wnvpresent, dtype: float64

In [5]:
x_train, x_test, y_train, y_test= train_test_split(x,y,random_state=42)

ss = StandardScaler()
x_train_sc = ss.fit_transform(x_train)
x_test_sc = ss.transform(x_test)

In [6]:
sm = SMOTE(random_state=42)

x_train_sc, y_train = sm.fit_resample(x_train_sc,y_train)

y_train.value_counts(normalize=True)

0    0.5
1    0.5
Name: wnvpresent, dtype: float64

In [13]:
# Instantiate models
models = {'lr': LogisticRegression(max_iter=5_000, random_state=42, solver='saga'),
          'rf': RandomForestClassifier(random_state=42),
          'gb': GradientBoostingClassifier(random_state=42),
          'et': ExtraTreesClassifier(random_state=42),
          'ada': AdaBoostClassifier(random_state=42),
          'svm': SVC(random_state=42, probability=True),
          'xgb': XGBClassifier(random_state=42)
}

In [8]:
# Function to run model -- input model
def get_model_scores(model_name,
                     mod, 
                     mod_params={}, 
                     grid_search=False):
    
    """Function accepts following inputs:
    Name of model (str), model to be used (str), 
    model params(dict, optional), grid_seach(boolean, optional)
    If grid_search is True, then please also input the relevant 
    params for GridSearching
    """
    
    # empty dict for appending results
    results = {}
    
    # instantiate pipe
    pipe = Pipeline([
            (mod, models[mod])
            ])
    
    # check if GridSearch true or false
    if grid_search:
        
        # combine vectorizer and mod params together
        gs_params = {}
        gs_params.update(mod_params)
        
        # instantiate GridSearchCV
        gs = GridSearchCV(pipe, 
                          param_grid=gs_params,
                          cv=5, 
                          verbose=2, 
                          n_jobs=-1)
        
        # fit model
        gs.fit(x_train_sc, y_train)
        pipe = gs
        
    else:
        # else fit model
        pipe.fit(x_train_sc, y_train)
    
    # create predictions and confusion matrix
    predictions = pipe.predict(x_test_sc)
    tn, fp, fn, tp = confusion_matrix(y_test, 
                                      predictions).ravel()
    y_test_pred_prob = pipe.predict_proba(x_test_sc)[:,1]
    y_train_pred_prob = pipe.predict_proba(x_train_sc)[:,1]

    train_auc = roc_auc_score(y_train, y_train_pred_prob)
    test_auc = roc_auc_score(y_test, y_test_pred_prob)
    
    # Retrieve metrics and add to results
    results['model_name'] = model_name
    results['model'] = mod
    results['train_score'] = pipe.score(x_train_sc, y_train)
    results['test_score'] = pipe.score(x_test_sc, y_test)
    
    results['recall'] = recall_score(y_test, 
                                     predictions)
    
    results['specificity'] = tn/(tn + fp)
    
    results['precision'] = precision_score(y_test, 
                                           predictions)
    
    results['train_auc'] = roc_auc_score(y_train, y_train_pred_prob)
    results['test_auc'] = roc_auc_score(y_test, y_test_pred_prob)
    
    results['is_tuned'] = grid_search
    
    if grid_search:
        print('BEST PARAMS-->')
        display(pipe.best_params_)
    
    # add results to list for model evaluation later
    model_eval.append(results)
    
    print('METRICS-->')
    display(results)
    
    tn, fp, fn, tp = confusion_matrix(y_test, 
                                      predictions).ravel()
    
    print(f"True Negatives: {tn}")
    print(f"False Positives: {fp}")
    print(f"False Negatives: {fn}")
    print(f"True Positives: {tp}")
    
    return pipe

In [9]:
# create list to store results
model_eval =[]

In [10]:
lr_base_multi = get_model_scores('lr_base_multi', 'lr')

METRICS-->


{'model_name': 'lr_base_multi',
 'model': 'lr',
 'train_score': 0.8259796806966618,
 'test_score': 0.775990099009901,
 'recall': 0.6048387096774194,
 'specificity': 0.7852173913043479,
 'precision': 0.13181019332161686,
 'train_auc': 0.8874044228083443,
 'test_auc': 0.7729926367461429,
 'is_tuned': False}

True Negatives: 1806
False Positives: 494
False Negatives: 49
True Positives: 75


In [11]:
svm_base_multi = get_model_scores('svm_base_multi', 'svm')

METRICS-->


{'model_name': 'svm_base_multi',
 'model': 'svm',
 'train_score': 0.9298984034833091,
 'test_score': 0.8535478547854786,
 'recall': 0.45161290322580644,
 'specificity': 0.8752173913043478,
 'precision': 0.16326530612244897,
 'train_auc': 0.9783968162352202,
 'test_auc': 0.777312412342216,
 'is_tuned': False}

True Negatives: 2013
False Positives: 287
False Negatives: 68
True Positives: 56


In [14]:
xg_base_multi = get_model_scores('xg_base_multi', 'xgb')

C:\Users\Jayy Jangam\.conda\envs\dsi\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[20:41:02] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
METRICS-->


{'model_name': 'xg_base_multi',
 'model': 'xgb',
 'train_score': 0.978011611030479,
 'test_score': 0.9422442244224423,
 'recall': 0.0967741935483871,
 'specificity': 0.9878260869565217,
 'precision': 0.3,
 'train_auc': 0.9974362836276467,
 'test_auc': 0.8528962131837308,
 'is_tuned': False}

True Negatives: 2272
False Positives: 28
False Negatives: 112
True Positives: 12


In [15]:
rf_base_multi = get_model_scores('rf_base_multi', 'rf')

METRICS-->


{'model_name': 'rf_base_multi',
 'model': 'rf',
 'train_score': 0.9918722786647315,
 'test_score': 0.9434818481848185,
 'recall': 0.07258064516129033,
 'specificity': 0.9904347826086957,
 'precision': 0.2903225806451613,
 'train_auc': 0.9997484522487946,
 'test_auc': 0.7970950210378681,
 'is_tuned': False}

True Negatives: 2278
False Positives: 22
False Negatives: 115
True Positives: 9


In [16]:
et_base_multi = get_model_scores('et_base_multi', 'et')

METRICS-->


{'model_name': 'et_base_multi',
 'model': 'et',
 'train_score': 0.9918722786647315,
 'test_score': 0.9426567656765676,
 'recall': 0.06451612903225806,
 'specificity': 0.99,
 'precision': 0.25806451612903225,
 'train_auc': 0.9998393056131918,
 'test_auc': 0.70117110799439,
 'is_tuned': False}

True Negatives: 2277
False Positives: 23
False Negatives: 116
True Positives: 8


In [17]:
gb_base_multi = get_model_scores('gb_base_multi', 'gb')

METRICS-->


{'model_name': 'gb_base_multi',
 'model': 'gb',
 'train_score': 0.9555152394775036,
 'test_score': 0.9377062706270627,
 'recall': 0.20161290322580644,
 'specificity': 0.9773913043478261,
 'precision': 0.3246753246753247,
 'train_auc': 0.9905579909041311,
 'test_auc': 0.8335729312762974,
 'is_tuned': False}

True Negatives: 2248
False Positives: 52
False Negatives: 99
True Positives: 25


In [18]:
ada_base_multi = get_model_scores('ada_base_multi', 'ada')

METRICS-->


{'model_name': 'ada_base_multi',
 'model': 'ada',
 'train_score': 0.9322931785195936,
 'test_score': 0.9129537953795379,
 'recall': 0.3870967741935484,
 'specificity': 0.941304347826087,
 'precision': 0.26229508196721313,
 'train_auc': 0.9820600415823189,
 'test_auc': 0.8415410238429173,
 'is_tuned': False}

True Negatives: 2165
False Positives: 135
False Negatives: 76
True Positives: 48


## Using the Regression Dataset

In [19]:
x = df_regression.drop(['wnvpresent'],axis=1)
y = df_regression['wnvpresent']

y.value_counts(normalize=True)

0    0.948107
1    0.051893
Name: wnvpresent, dtype: float64

In [20]:
x_train, x_test, y_train, y_test= train_test_split(x,y,random_state=42)

ss = StandardScaler()
x_train_sc = ss.fit_transform(x_train)
x_test_sc = ss.transform(x_test)

In [21]:
sm = SMOTE(random_state=42)

x_train_sc, y_train = sm.fit_resample(x_train_sc,y_train)

y_train.value_counts(normalize=True)

0    0.5
1    0.5
Name: wnvpresent, dtype: float64

---

In [22]:
lr_base_reg = get_model_scores('lr_base_reg', 'lr')

METRICS-->


{'model_name': 'lr_base_reg',
 'model': 'lr',
 'train_score': 0.8057329462989841,
 'test_score': 0.7541254125412541,
 'recall': 0.5887096774193549,
 'specificity': 0.7630434782608696,
 'precision': 0.11812297734627832,
 'train_auc': 0.8751527212826059,
 'test_auc': 0.7461448106591865,
 'is_tuned': False}

True Negatives: 1755
False Positives: 545
False Negatives: 51
True Positives: 73


In [23]:
svm_base_reg = get_model_scores('svm_base_reg', 'svm')

METRICS-->


{'model_name': 'svm_base_reg',
 'model': 'svm',
 'train_score': 0.9130624092888244,
 'test_score': 0.8407590759075908,
 'recall': 0.4274193548387097,
 'specificity': 0.8630434782608696,
 'precision': 0.14402173913043478,
 'train_auc': 0.9712810577160058,
 'test_auc': 0.7514708976157083,
 'is_tuned': False}

True Negatives: 1985
False Positives: 315
False Negatives: 71
True Positives: 53


In [24]:
xg_base_reg = get_model_scores('xg_base_reg', 'xgb')

[20:44:28] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\Jayy Jangam\.conda\envs\dsi\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


METRICS-->


{'model_name': 'xg_base_reg',
 'model': 'xgb',
 'train_score': 0.9776487663280116,
 'test_score': 0.9426567656765676,
 'recall': 0.08064516129032258,
 'specificity': 0.9891304347826086,
 'precision': 0.2857142857142857,
 'train_auc': 0.9971802490304831,
 'test_auc': 0.8572072230014025,
 'is_tuned': False}

True Negatives: 2275
False Positives: 25
False Negatives: 114
True Positives: 10


In [25]:
rf_base_reg = get_model_scores('rf_base_reg', 'rf')

METRICS-->


{'model_name': 'rf_base_reg',
 'model': 'rf',
 'train_score': 0.9918722786647315,
 'test_score': 0.9418316831683168,
 'recall': 0.06451612903225806,
 'specificity': 0.9891304347826086,
 'precision': 0.24242424242424243,
 'train_auc': 0.9997510643093522,
 'test_auc': 0.7902559607293127,
 'is_tuned': False}

True Negatives: 2275
False Positives: 25
False Negatives: 116
True Positives: 8


In [26]:
et_base_reg = get_model_scores('et_base_reg', 'et')

METRICS-->


{'model_name': 'et_base_reg',
 'model': 'et',
 'train_score': 0.9918722786647315,
 'test_score': 0.941006600660066,
 'recall': 0.07258064516129033,
 'specificity': 0.9878260869565217,
 'precision': 0.24324324324324326,
 'train_auc': 0.9998393056131918,
 'test_auc': 0.6866374474053296,
 'is_tuned': False}

True Negatives: 2272
False Positives: 28
False Negatives: 115
True Positives: 9


In [27]:
gb_base_reg = get_model_scores('gb_base_reg', 'gb')

METRICS-->


{'model_name': 'gb_base_reg',
 'model': 'gb',
 'train_score': 0.9613207547169811,
 'test_score': 0.9389438943894389,
 'recall': 0.12903225806451613,
 'specificity': 0.9826086956521739,
 'precision': 0.2857142857142857,
 'train_auc': 0.9908925453055584,
 'test_auc': 0.8262903225806452,
 'is_tuned': False}

True Negatives: 2260
False Positives: 40
False Negatives: 108
True Positives: 16


In [28]:
ada_base_reg = get_model_scores('ada_base_reg', 'ada')

METRICS-->


{'model_name': 'ada_base_reg',
 'model': 'ada',
 'train_score': 0.9402757619738752,
 'test_score': 0.9261551155115512,
 'recall': 0.18548387096774194,
 'specificity': 0.9660869565217391,
 'precision': 0.22772277227722773,
 'train_auc': 0.9831208330787979,
 'test_auc': 0.8237394810659187,
 'is_tuned': False}

True Negatives: 2222
False Positives: 78
False Negatives: 101
True Positives: 23


Now let's evaluate all our models and see which model, and which dataset has performed the best. Based on this, we will decide which models to further tune.<br>
The main metrics we are looking for are `recall` and `test_auc`.

In [34]:
model_scores = pd.DataFrame(model_eval)

In [35]:
model_scores.sort_values(by=['recall', 'test_auc'], ascending=False)[['model_name', 'model', 
                                                                          'recall', 'test_auc']]

,model_name,model,recall,test_auc
0,lr_base_multi,lr,0.604839,0.772993
7,lr_base_reg,lr,0.588710,0.746145
1,svm_base_multi,svm,0.451613,0.777312
8,svm_base_reg,svm,0.427419,0.751471
6,ada_base_multi,ada,0.387097,0.841541
5,gb_base_multi,gb,0.201613,0.833573
13,ada_base_reg,ada,0.185484,0.823739
12,gb_base_reg,gb,0.129032,0.826290
2,xg_base_multi,xgb,0.096774,0.852896
9,xg_base_reg,xgb,0.080645,0.857207


Since our scores are better using the dataset with features having multi-collinearity, we will continue using that dataset moving foward.

In [36]:
model_scores.to_csv('../data/base_model_scores.csv')

In [28]:
lr = LogisticRegression()

lr_scores = cross_val_score(lr, x_train_sc , y_train, cv=5)

lr.fit(x_train_sc,y_train)

display(np.mean(lr_scores))

display(lr.score(x_train_sc,y_train))

display(lr.score(x_test_sc,y_test))

0.9569264955894103

0.9585867361714451

0.9306930693069307

In [39]:
predictions = lr.predict(x_test_sc)
tn, fp, fn, tp = confusion_matrix(y_test, predictions).ravel()
y_test_pred_prob = lr.predict_proba(x_test_sc)[:,1]
y_train_pred_prob = lr.predict_proba(x_train_sc)[:,1]

train_auc = roc_auc_score(y_train, y_train_pred_prob)
test_auc = roc_auc_score(y_test, y_test_pred_prob)

display(train_auc)
display(test_auc)

In [44]:
svm = SVC(probability=True)

svm_scores = cross_val_score(svm, x_train_sc , y_train, cv=5)

svm.fit(x_train_sc,y_train)

display(np.mean(svm_scores))

display(svm.score(x_train_sc,y_train))

display(svm.score(x_test_sc,y_test))

0.9556238522757143

0.9614827686070084

0.9352310231023102

In [45]:
predictions = svm.predict(x_test_sc)
tn, fp, fn, tp = confusion_matrix(y_test, predictions).ravel()
y_test_pred_prob = svm.predict_proba(x_test_sc)[:,1]
y_train_pred_prob = svm.predict_proba(x_train_sc)[:,1]

train_auc = roc_auc_score(y_train, y_train_pred_prob)
test_auc = roc_auc_score(y_test, y_test_pred_prob)

display(train_auc)
display(test_auc)

0.9937922123146546

0.7117353640230173